In [85]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import operator
from math import log
from collections import Counter
from statistics import mean
from sklearn.preprocessing import LabelEncoder 

class CustomANN:
  def __init__(self,input_size,hidden_layers,output_size,learning_rate):
    self.input_node_size=input_size
    self.output_node_size=output_size
    self.hidden_layers=hidden_layers
    self.learning_rate=learning_rate

    self.weights=[]
    self.biases=[]
    layer_sizes=[input_size]+hidden_layers+[output_size]

    for i in range(len(layer_sizes)-1):
      self.weights.append(np.random.randn(layer_sizes[i],layer_sizes[i+1]))
      self.biases.append(np.random.randn(layer_sizes[i+1]))

  def relu(x):
     if x > 0:
        x = x
     else:
        x = 0
     return x

  def relu_derivative(x):
     if x > 0:
        return 1
     else:
        return 0

  def forward(self, x):
      
      activations = [x]
      for i in range(len(self.weights)):
         z = np.dot(activations[-1], self.weights[i]) + self.biases[i]
         a = self.relu(z) if i < len(self.weights) - 1 else z  
         activations.append(a)
      return activations

  def backward(self, activations, y):    
      delta = (activations[-1] - y).reshape(1, -1)  
      for i in reversed(range(len(self.weights))):
         a = activations[i].reshape(1, -1)  
         if i > 0:
               delta = np.dot(delta, self.weights[i].T) * self.relu_derivative(a)
               
         dW = np.dot(a.T, delta) 
         dB = np.sum(delta, axis=0, keepdims=True)
         print(self.weights[i].shape)
         print((self.learning_rate * dW.T).shape)

            
         resized_dW = np.resize(self.learning_rate * dW.T, (1, dW.shape[0])).T
         print(resized_dW.shape, self.weights[i].shape)
         self.weights[i] -= resized_dW  
         
         print(self.biases[i].shape,(self.learning_rate * dB.squeeze()).shape)
         resized_dB = np.resize(self.learning_rate * dB.squeeze(), (1, 1)).T
         resized_self_biases = np.resize(self.biases[i], (1, 1)).T
         resized_self_biases -= resized_dB  

  def train(self, X, y):
      activations = self.forward(X)
      self.backward(activations, y)
   
  def test(self,X):
      activations=self.forward(X)
      return activations[-1]

df = pd.read_csv('./student-mat.csv',sep=";")
df.head(5)
df.dropna()
df.head(5)

X = df.drop(['G1', 'G2', 'G3'],axis=1)
y = df[['G1', 'G2', 'G3']]

label_encode = LabelEncoder()
for column in y.columns:
   y[column] = label_encode.fit_transform(y[column])
y = y.astype(float)

label_encode = LabelEncoder()
for column in X.columns:
   X[column] = label_encode.fit_transform(X[column])
X = X.astype(float)

df = pd.get_dummies(df, columns = df.columns) 
X_train,  X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,shuffle=True)

model=CustomANN(30,[],3,0.01)

for i in range(100): 
  for j in range(X_train.shape[0]):
     x = np.array(X_train.iloc[j].values.reshape(1, -1))
     y_true = np.array(y_train.iloc[j].values.reshape(1, -1))
     model.train(x, y_true)

y_pred=[]

for j in range(X_test.shape[0]):
   x = np.array(X_test.iloc[j].values.reshape(1, -1))
   y_pred.append(model.test(x))
y_pred=np.array(y_pred)

print("y_pred: ",y_pred)

C:\Users\ishaa\AppData\Local\Temp\ipykernel_2828\1371535836.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[column] = label_encode.fit_transform(y[column])
C:\Users\ishaa\AppData\Local\Temp\ipykernel_2828\1371535836.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[column] = label_encode.fit_transform(y[column])
C:\Users\ishaa\AppData\Local\Temp\ipykernel_2828\1371535836.py:89: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3,) (3,)
(30, 3)
(3, 30)
(30, 1) (30, 3)
(3